In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyrender
import os
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torch.autograd import Variable
from torchsummary import summary
import skimage.measure as sk

import h5py

import time
import pymesh
import trimesh

#%matplotlib notebook

In [3]:
device = torch.device("cuda")

# reproducible.
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
torch.manual_seed(0)

In [4]:
device

device(type='cuda')

In [5]:
class ChairDepthDataset(Dataset):
    
    def __init__(self, h5_file):
        
        self.hf = h5py.File(h5_file, 'r')
        self.keys = list(self.hf.keys())
        
        self.to_tensor = transforms.ToTensor()

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        
        group = self.hf[self.keys[idx]]
        
        model_id = str(self.keys[idx])
        depth_img_raw = self.to_tensor(Image.fromarray(np.array(group['depth_img'])))
        azimuth = float(group['azimuth'][()])
        elevation = float(group['elevation'][()])
        distance = float(group['distance'][()])
        target_vox = torch.tensor(group['target_vox'], dtype=torch.float)
        
        depth_img = nn.functional.interpolate(depth_img_raw.unsqueeze(0), size=256, mode='nearest').squeeze(0)
        
        sample = {'model_id': model_id,
                  'depth_img_raw': depth_img_raw,
                  'depth_img': depth_img,
                  'azimuth': azimuth,
                  'elevation': elevation,
                  'distance': distance,
                  'target_vox': target_vox, 
                 }

        return sample

In [6]:
data_file = '/home/ankbzpx/datasets/ShapeNet/ShapeNetRenderingh5_v1/03001627/data_test_rescale.h5'

test_depth_dataset = ChairDepthDataset(data_file)

In [7]:
# plot vox or img
from vis_utils import plotFromVoxels, plotImg, plot_image_list
# generate with continuous model
from render_utils import generate_mesh, get_relative_transform_matrix
# render mesh with pyrender
from render_utils import render
# mesh from ground truth vox
from render_utils import RotateAlongAxis
# transfrom sdf
from render_utils import get_transformed_indices, sdf2Voxel, get_meshgrid, get_transformed_meshgrid, get_relative_transformed_vox
# get cd, emd, iou from 2 pymesh
from render_utils import get_test_results

In [8]:
# Pre-trained Model path
MODEL_PATH = '/home/ankbzpx/Documents/'
discrete_encoder_path = MODEL_PATH + 'discrete continuous shape/discrete_encoder_model_256.pth'
discrete_decoder_path = MODEL_PATH + 'discrete continuous shape/discrete_decoder_model_256.pth'
unet_path = MODEL_PATH + 'discrete continuous shape/con_unet.pth'
continuous_model_path = MODEL_PATH + 'discrete continuous shape/continuous_model_dropout.pth'
mapping_path = MODEL_PATH + 'discrete continuous shape/mapping_256.pth'


from models import Discrete_encoder, Discrete_decoder, Conditional_UNET, Continuous, Mapping

####################
# Discrete Encoder #
####################

discrete_encoder = Discrete_encoder(256).to(device)
discrete_encoder.load_state_dict(torch.load(discrete_encoder_path))
discrete_encoder.eval()

for child in discrete_encoder.children():
    for param in child.parameters():
        param.requires_grad = False
        
###########
# Mapping #
###########
        
mapping = Mapping(256).to(device)
mapping.load_state_dict(torch.load(mapping_path))
e = mapping.eval()

for child in mapping.children():
    for param in child.parameters():
        param.requires_grad = False        
        

####################
# Discrete Decoder #
####################
        
discrete_decoder = Discrete_decoder(256).to(device)
discrete_decoder.load_state_dict(torch.load(discrete_decoder_path))
discrete_decoder.eval()

# for child in discrete_decoder.children():
#     for param in child.parameters():
#         param.requires_grad = False
        
########
# UNET #
########

# pre-trained model is loaded within the model
unet = Conditional_UNET(unet_path).to(device)

unet.eval()

for child in unet.children():
    for param in child.parameters():
        param.requires_grad = False


        
##############
# Continuous #
##############

continuous = Continuous().to(device)
continuous.load_state_dict(torch.load(continuous_model_path))
continuous.eval()

for child in continuous.children():
    for param in child.parameters():
        param.requires_grad = False

In [9]:
voxsize = 32
D2R = np.pi/180.0

In [10]:
# custom depth consistency loss

from torch.autograd import Function
from torch.autograd.function import once_differentiable

class DepthConsistencyLoss(Function):

    @staticmethod
    def forward(ctx, out_vox, depth_index, close_index):
        
        # out_vox shape n x 1 x 32 x 32 x 32
        # depth_index shape n x 32 x 32 x 32
        # close_index shape n x 32 x 32 x 32
        
        ctx.save_for_backward(out_vox)
        
        vox = out_vox.squeeze(1)
        ctx.depth_index = depth_index
        ctx.close_index = close_index
        
        loss = torch.zeros_like(depth_index, dtype=torch.float).to(device)
        
        #bce form
        loss[depth_index.bool()] = -torch.log(vox[depth_index.bool()])
        loss[close_index.bool()] = -torch.log(1-vox[close_index.bool()])
        
        return torch.sum(loss) / (torch.sum(depth_index) + torch.sum(close_index))
        
    @staticmethod
    @once_differentiable
    def backward(ctx, grad_output):
        
        out_vox, = ctx.saved_tensors
        
        vox = out_vox.squeeze(1)
        grad = torch.zeros_like(ctx.depth_index, dtype=torch.float).to(device)
        
        #bce form
        grad[ctx.depth_index.bool()] = -1/vox[ctx.depth_index.bool()]
        grad[ctx.close_index.bool()] = 1/(1-vox[ctx.close_index.bool()])
        
        return grad.unsqueeze(1), None, None, None

In [11]:
from refine_utils import get_radius, get_depth_close_idx

In [12]:
from render_utils import mesh_from_voxel

In [13]:
%matplotlib notebook

mesh_grid_res = 64
mesh_batch_size = 128

def depth_refine_three_imgs(idx, lr = 3e-4, iter_count = 30):
    
    data_from = test_depth_dataset[24*idx]
    depth_img_from = data_from['depth_img'].unsqueeze(0)
    depth_img_raw_from = data_from['depth_img_raw'].unsqueeze(0)
    target_vox_from = data_from['target_vox']
    distance_from = data_from['distance']
    azimuth_from = data_from['azimuth']
    elevation_from = data_from['elevation']
    
    #plotFromVoxels(target_vox_from[0])
    
    ########### initial depth ##########
    
    radius = get_radius(depth_img_raw_from, distance_from)
    depth_index_from, close_index_from = get_depth_close_idx(depth_img_raw_from, distance_from, radius)
    
    ########### gt mesh ##########
    
    model_dir = path_model + test_depth_dataset[24*idx]['model_id'].split('_')[0] + '/model.obj'
    mesh_py = pymesh.load_mesh(model_dir)
    transformed_vertices = get_transformed_indices(mesh_py.vertices, azimuth_from, elevation_from, 1)
    gt_radius = np.max(np.linalg.norm(transformed_vertices, axis = 1))
    mesh_gt = pymesh.form_mesh(transformed_vertices/gt_radius, mesh_py.faces)
    
    #plotImg(depth_img_from[0, 0].detach().cpu().numpy())
    
    z = discrete_encoder(depth_img_from.to(device))
    w = mapping(z)
    w.requires_grad = True
    
    optimizer = torch.optim.SGD([w], lr=lr, momentum=0.5)
    
    out_vox = torch.sigmoid(discrete_decoder(w))
    #plotFromVoxels(out_vox[0, 0].detach().cpu().numpy() > 0.5)
    
    mesh_vox_before = mesh_from_voxel(out_vox[0, 0])
    
    mesh_before = generate_mesh(continuous, unet, out_vox, z, device, vox_res = 32, grid_res = mesh_grid_res, batch_size = mesh_batch_size, 
                                azimuth = 0, elevation = 0, isosurface = 0.0)
    
    ran = np.random.choice(np.arange(1, 24), 2)
    
    ########### first depth ##########
    
    idx_to_1 = 24*idx + ran[0]
    data_to_1 = test_depth_dataset[idx_to_1]
    depth_img_raw_to_1 = data_to_1['depth_img_raw'].unsqueeze(0)
    distance_to_1 = data_to_1['distance']
    azimuth_to_1 = data_to_1['azimuth']
    elevation_to_1 = data_to_1['elevation']
    depth_index_to_1, close_index_to_1 = get_depth_close_idx(depth_img_raw_to_1, distance_to_1, radius)
    
    #plotImg(depth_img_raw_to_1[0, 0].detach().cpu().numpy())
    
    ########### second depth ##########
    
    idx_to_2 = 24*idx + ran[1]
    data_to_2 = test_depth_dataset[idx_to_2]
    depth_img_raw_to_2 = data_to_2['depth_img_raw'].unsqueeze(0)
    distance_to_2 = data_to_2['distance']
    azimuth_to_2 = data_to_2['azimuth']
    elevation_to_2 = data_to_2['elevation']
    depth_index_to_2, close_index_to_2 = get_depth_close_idx(depth_img_raw_to_2, distance_to_2, radius)
    
    #plotImg(depth_img_raw_to_2[0, 0].detach().cpu().numpy())
    
    # Optimize with depth consistency loss
    for i in range(iter_count):
        
        optimizer.zero_grad()
        
        
        relative_transformed_vox_1 = get_relative_transformed_vox(out_vox, -azimuth_from, -elevation_from, azimuth_to_1, elevation_to_1, 
                                                            device, voxsize = 32, align_mode = 'zeros')
        
        relative_transformed_vox_2 = get_relative_transformed_vox(out_vox, -azimuth_from, -elevation_from, azimuth_to_2, elevation_to_2, 
                                                            device, voxsize = 32, align_mode = 'zeros')
        
        loss_0 = DepthConsistencyLoss.apply(out_vox, depth_index_from.unsqueeze(0).to(device), close_index_from.unsqueeze(0).to(device))
        loss_1 = DepthConsistencyLoss.apply(relative_transformed_vox_1, depth_index_to_1.unsqueeze(0).to(device), close_index_to_1.unsqueeze(0).to(device))
        loss_2 = DepthConsistencyLoss.apply(relative_transformed_vox_2, depth_index_to_2.unsqueeze(0).to(device), close_index_to_2.unsqueeze(0).to(device))
        
        loss = 2*loss_0 + loss_1 + loss_2
        
#         if i == 0:
#             print("Initial loss: ", loss.item())
#         else:
#             print(loss.item())
        
        loss.backward(retain_graph=True)
        optimizer.step()
        
        out_vox = torch.sigmoid(discrete_decoder(w))
        
    #plotFromVoxels(out_vox[0, 0].detach().cpu().numpy() > 0.5)    
    
    mesh_after = generate_mesh(continuous, unet, out_vox, z, device, vox_res = 32, grid_res = mesh_grid_res, batch_size = mesh_batch_size, 
                               azimuth = 0, elevation = 0, isosurface = 0.0)
    
    mesh_vox_after = mesh_from_voxel(out_vox[0, 0])
    
    return mesh_gt, mesh_vox_before, mesh_vox_after, mesh_before, mesh_after

In [14]:
def normalize_mesh(mesh_py, vox_size = 64):
    
    normalized_vertices = mesh_py.vertices - (vox_size/2)
    normalized_vertices /= np.max(np.linalg.norm(normalized_vertices, axis = 1))
    
    return pymesh.form_mesh(normalized_vertices, mesh_py.faces)

In [49]:
path_model = '/home/ankbzpx/datasets/ShapeNet/ShapeNetCore.v1/03001627/'

cd_gt_v_b_list = []
emd_gt_v_b_list = []
iou_gt_v_b_list = []

cd_gt_v_a_list = []
emd_gt_v_a_list = []
iou_gt_v_a_list = []

cd_gt_m_b_list = []
emd_gt_m_b_list = []
iou_gt_m_b_list = []

cd_gt_m_a_list = []
emd_gt_m_a_list = []
iou_gt_m_a_list = []

failed_idx = []

# int(len(test_depth_dataset)/24)
for idx in range(int(len(test_depth_dataset)/24)):
    
    print(idx)
    start_time = time.time()
    
    mesh_gt, mesh_vox_before, mesh_vox_after, mesh_before, mesh_after = depth_refine_three_imgs(idx)
    
    if mesh_gt is None or mesh_vox_before is None or mesh_vox_after is None or mesh_before is None or mesh_after is None:
        print('Failed case')
        failed_idx.append(idx)
        continue
        
    #mesh_gt = normalize_mesh(mesh_gt)
    mesh_vox_before = normalize_mesh(mesh_vox_before, 64)
    mesh_vox_after = normalize_mesh(mesh_vox_after, 64)
    mesh_before = normalize_mesh(mesh_before, mesh_grid_res)
    mesh_after = normalize_mesh(mesh_after, mesh_grid_res)
    
    cd_gt_v_b, emd_gt_v_b, iou_gt_v_b = get_test_results(mesh_gt, mesh_vox_before)
    cd_gt_v_a, emd_gt_v_a, iou_gt_v_a = get_test_results(mesh_gt, mesh_vox_after)
    cd_gt_m_b, emd_gt_m_b, iou_gt_m_b = get_test_results(mesh_gt, mesh_before)
    cd_gt_m_a, emd_gt_m_a, iou_gt_m_a = get_test_results(mesh_gt, mesh_after)
    
    cd_gt_v_b_list.append(cd_gt_v_b)
    emd_gt_v_b_list.append(emd_gt_v_b)
    iou_gt_v_b_list.append(iou_gt_v_b)
    
    cd_gt_v_a_list.append(cd_gt_v_a)
    emd_gt_v_a_list.append(emd_gt_v_a)
    iou_gt_v_a_list.append(iou_gt_v_a)
    
    cd_gt_m_b_list.append(cd_gt_m_b)
    emd_gt_m_b_list.append(emd_gt_m_b)
    iou_gt_m_b_list.append(iou_gt_m_b)
    
    cd_gt_m_a_list.append(cd_gt_m_a)
    emd_gt_m_a_list.append(emd_gt_m_a)
    iou_gt_m_a_list.append(iou_gt_m_a)
    
    print("--- %s seconds ---" % (time.time() - start_time))

0
--- 7.124056339263916 seconds ---
1
--- 5.5772998332977295 seconds ---
2
--- 5.618065118789673 seconds ---
3
--- 6.284072637557983 seconds ---
4
--- 5.342178106307983 seconds ---
5
--- 5.133547067642212 seconds ---
6
--- 5.047257661819458 seconds ---
7
--- 5.322369813919067 seconds ---
8
--- 4.9053215980529785 seconds ---
9
--- 5.7420759201049805 seconds ---
10
--- 8.021786451339722 seconds ---
11
--- 5.874244928359985 seconds ---
12
--- 6.135941505432129 seconds ---
13
--- 5.472238063812256 seconds ---
14
--- 5.18369460105896 seconds ---
15
--- 4.694316864013672 seconds ---
16
--- 6.094900131225586 seconds ---
17
--- 7.160055637359619 seconds ---
18
--- 5.759783983230591 seconds ---
19
--- 5.4855265617370605 seconds ---
20
--- 7.137930154800415 seconds ---
21
--- 5.216404438018799 seconds ---
22
--- 5.217291831970215 seconds ---
23
--- 5.268719911575317 seconds ---
24
--- 5.6297948360443115 seconds ---
25
--- 5.351110219955444 seconds ---
26
--- 5.688872575759888 seconds ---
27
--- 

--- 8.009704351425171 seconds ---
219
--- 5.425712585449219 seconds ---
220
--- 5.204500675201416 seconds ---
221
--- 5.103608131408691 seconds ---
222
--- 6.376535177230835 seconds ---
223
--- 5.699768304824829 seconds ---
224
--- 5.618597745895386 seconds ---
225
--- 5.162671804428101 seconds ---
226
--- 6.704237222671509 seconds ---
227
--- 5.47980523109436 seconds ---
228
--- 5.122592926025391 seconds ---
229
--- 5.406397819519043 seconds ---
230
--- 5.86539888381958 seconds ---
231
--- 5.0995659828186035 seconds ---
232
--- 5.973625898361206 seconds ---
233
--- 5.041698217391968 seconds ---
234
--- 5.434004545211792 seconds ---
235
--- 5.337867736816406 seconds ---
236
--- 5.3817808628082275 seconds ---
237
--- 5.071553707122803 seconds ---
238
--- 5.24417781829834 seconds ---
239
--- 5.448342800140381 seconds ---
240
--- 5.26058292388916 seconds ---
241
--- 5.268025636672974 seconds ---
242
--- 5.065555095672607 seconds ---
243
--- 5.41539192199707 seconds ---
244
--- 5.189138174

--- 5.488907814025879 seconds ---
435
--- 5.459676504135132 seconds ---
436
--- 5.747684478759766 seconds ---
437
--- 7.600625038146973 seconds ---
438
--- 6.801096200942993 seconds ---
439
--- 5.205527305603027 seconds ---
440
--- 6.014551877975464 seconds ---
441
--- 8.174844026565552 seconds ---
442
--- 5.352802038192749 seconds ---
443
--- 5.572526693344116 seconds ---
444
--- 5.915429353713989 seconds ---
445
--- 6.559471607208252 seconds ---
446
--- 6.275466680526733 seconds ---
447
--- 6.4448606967926025 seconds ---
448
--- 7.405657768249512 seconds ---
449
--- 5.364206314086914 seconds ---
450
--- 5.210241794586182 seconds ---
451
--- 6.9042136669158936 seconds ---
452
--- 5.70358681678772 seconds ---
453
--- 6.128492593765259 seconds ---
454
--- 6.031495571136475 seconds ---
455
--- 7.0284583568573 seconds ---
456
--- 5.60347843170166 seconds ---
457
--- 7.227686166763306 seconds ---
458
--- 5.162691593170166 seconds ---
459
--- 5.366348505020142 seconds ---
460
--- 5.99610853

--- 6.588472843170166 seconds ---
651
--- 5.312962293624878 seconds ---
652
--- 5.693641662597656 seconds ---
653
--- 5.277766704559326 seconds ---
654
--- 5.356321334838867 seconds ---
655
--- 6.34016227722168 seconds ---
656
--- 5.892747640609741 seconds ---
657
--- 5.260295629501343 seconds ---
658
--- 7.183003187179565 seconds ---
659
--- 5.620388746261597 seconds ---
660
--- 6.574864625930786 seconds ---
661
--- 7.808237314224243 seconds ---
662
--- 7.810741901397705 seconds ---
663
--- 5.81331467628479 seconds ---
664
--- 5.57616114616394 seconds ---
665
--- 5.22933292388916 seconds ---
666
--- 5.180614471435547 seconds ---
667
--- 5.188912630081177 seconds ---
668
--- 6.074935674667358 seconds ---
669
--- 5.20741605758667 seconds ---
670
--- 5.77545428276062 seconds ---
671
--- 5.7570507526397705 seconds ---
672
--- 5.191689968109131 seconds ---
673
--- 5.741328001022339 seconds ---
674
--- 5.633009672164917 seconds ---
675
--- 5.954219341278076 seconds ---
676
--- 5.41475749015

--- 5.85880446434021 seconds ---
867
--- 5.526343822479248 seconds ---
868
--- 5.713102340698242 seconds ---
869
--- 5.87436842918396 seconds ---
870
--- 7.487943887710571 seconds ---
871
--- 6.461263656616211 seconds ---
872
--- 5.419842720031738 seconds ---
873
--- 5.194383859634399 seconds ---
874
--- 5.692846059799194 seconds ---
875
--- 7.173944711685181 seconds ---
876
--- 7.626030206680298 seconds ---
877
--- 5.208547115325928 seconds ---
878
--- 5.3133580684661865 seconds ---
879
--- 5.700772285461426 seconds ---
880
--- 5.772124290466309 seconds ---
881
--- 5.299111604690552 seconds ---
882
--- 5.492275953292847 seconds ---
883
--- 6.554135084152222 seconds ---
884
--- 5.991253852844238 seconds ---
885
--- 7.59762978553772 seconds ---
886
--- 5.589093208312988 seconds ---
887
--- 5.089176416397095 seconds ---
888
--- 5.121343374252319 seconds ---
889
--- 5.435212850570679 seconds ---
890
--- 6.166461706161499 seconds ---
891
--- 6.1164727210998535 seconds ---
892
--- 5.6977097

--- 5.1525702476501465 seconds ---
1081
--- 5.200999975204468 seconds ---
1082
--- 5.370387554168701 seconds ---
1083
--- 7.174891710281372 seconds ---
1084
--- 5.098156690597534 seconds ---
1085
--- 6.4911417961120605 seconds ---
1086
--- 5.319756031036377 seconds ---
1087
--- 5.5164430141448975 seconds ---
1088
--- 5.199962615966797 seconds ---
1089
--- 5.95254111289978 seconds ---
1090
--- 5.33735466003418 seconds ---
1091
--- 7.7237536907196045 seconds ---
1092
--- 5.289404392242432 seconds ---
1093
--- 5.085948467254639 seconds ---
1094
--- 5.358933210372925 seconds ---
1095
--- 5.70750093460083 seconds ---
1096
--- 6.623829364776611 seconds ---
1097
--- 6.098771572113037 seconds ---
1098
--- 5.785116672515869 seconds ---
1099
--- 5.553410053253174 seconds ---
1100
--- 6.650493860244751 seconds ---
1101
--- 6.089166164398193 seconds ---
1102
--- 5.3756444454193115 seconds ---
1103
--- 5.2660911083221436 seconds ---
1104
--- 5.289307117462158 seconds ---
1105
--- 7.254791259765625 

--- 5.338287830352783 seconds ---
1291
--- 6.445303916931152 seconds ---
1292
--- 7.15781307220459 seconds ---
1293
--- 5.643297433853149 seconds ---
1294
--- 5.230565071105957 seconds ---
1295
--- 5.222235441207886 seconds ---
1296
--- 5.074472427368164 seconds ---
1297
--- 5.931053638458252 seconds ---
1298
--- 5.58735728263855 seconds ---
1299
--- 6.905272960662842 seconds ---
1300
--- 8.178989171981812 seconds ---
1301
--- 5.759930849075317 seconds ---
1302
--- 5.317728519439697 seconds ---
1303
--- 5.663097858428955 seconds ---
1304
--- 6.547122478485107 seconds ---
1305
--- 5.50067925453186 seconds ---
1306
--- 5.345625638961792 seconds ---
1307
--- 5.3623206615448 seconds ---
1308
--- 5.9108617305755615 seconds ---
1309
--- 6.070186138153076 seconds ---
1310
--- 5.300572633743286 seconds ---


In [23]:
render(mesh_gt)

In [40]:
render(mesh_vox_before)

In [41]:
render(mesh_vox_after)

In [46]:
render(mesh_before)

In [52]:
render(mesh_after)

In [51]:
print('--- vox before ---')
print('Chamfer Distacne: m_', np.mean(cd_gt_v_b_list), ' s_', np.std(cd_gt_v_b_list))
print()
print('Earth Movers Distance: m_', np.mean(emd_gt_v_b_list), ' s_', np.std(emd_gt_v_b_list))
print()
print('Intersection over Union: m_', np.mean(iou_gt_v_b_list), ' s_', np.std(iou_gt_v_b_list))
print()

print('--- vox after ---')
print('Chamfer Distacne: m_', np.mean(cd_gt_v_a_list), ' s_', np.std(cd_gt_v_a_list))
print()
print('Earth Movers Distance: m_', np.mean(emd_gt_v_a_list), ' s_', np.std(emd_gt_v_a_list))
print()
print('Intersection over Union: m_', np.mean(iou_gt_v_a_list), ' s_', np.std(iou_gt_v_a_list))
print()

print('--- mesh before ---')
print('Chamfer Distacne: m_', np.mean(cd_gt_m_b_list), ' s_', np.std(cd_gt_m_b_list))
print()
print('Earth Movers Distance: m_', np.mean(emd_gt_m_b_list), ' s_', np.std(emd_gt_m_b_list))
print()
print('Intersection over Union: m_', np.mean(iou_gt_m_b_list), ' s_', np.std(iou_gt_m_b_list))
print()

print('--- mesh after ---')
print('Chamfer Distacne: m_', np.mean(cd_gt_m_a_list), ' s_', np.std(cd_gt_m_a_list))
print()
print('Earth Movers Distance: m_', np.mean(emd_gt_m_a_list), ' s_', np.std(emd_gt_m_a_list))
print()
print('Intersection over Union m_', np.mean(iou_gt_m_a_list), ' s_', np.std(iou_gt_m_a_list))
print()

--- vox before ---
Chamfer Distacne: m_ 0.0286311  s_ 0.02186652

Earth Movers Distance: m_ 0.042188884914124994  s_ 0.019863599850553644

Intersection over Union: m_ 0.50869310720466  s_ 0.13626359533954435

--- vox after ---
Chamfer Distacne: m_ 0.025010476  s_ 0.0096005965

Earth Movers Distance: m_ 0.04357991534053904  s_ 0.016240417824117965

Intersection over Union: m_ 0.5238620315256167  s_ 0.11551493523142507

--- mesh before ---
Chamfer Distacne: m_ 0.02442473  s_ 0.011893644

Earth Movers Distance: m_ 0.03176168753703321  s_ 0.01771753326149935

Intersection over Union: m_ 0.5122154073220214  s_ 0.15145497369211805

--- mesh after ---
Chamfer Distacne: m_ 0.024866318  s_ 0.011352454

Earth Movers Distance: m_ 0.036105982693745445  s_ 0.018651543036001184

Intersection over Union m_ 0.5086855361424986  s_ 0.1545311396507649



In [189]:
int(len(test_depth_dataset)/24)

1311